### 8. 감성분석 IMDb model


In [2]:
import pyprind
import pandas as pd
import os
import sys

basepath = 'data/aclImdb'
labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)

data = []  # 데이터를 리스트에 저장
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
                data.append([txt, labels[l]])  # 데이터를 리스트에 추가
                pbar.update()

# 리스트 데이터를 한 번에 DataFrame으로 변환
df = pd.DataFrame(data, columns=['review', 'sentiment'])

# 확인용 출력
print(df.head())


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02
                                              review  sentiment
0  I went and saw this movie last night after bei...          1
1  Actor turned director Bill Paxton follows up h...          1
2  As a recreational golfer with some knowledge o...          1
3  I saw this film in a sneak preview, and it is ...          1
4  Bill Paxton has taken the true story of the 19...          1


In [7]:
# data save as .csv
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('data/movie_data.csv',index = False, encoding = 'utf-8')

In [12]:
# read data
df = pd.read_csv('data/movie_data.csv', encoding = 'utf-8')
df.head(3)
df.shape

(50000, 2)

In [21]:
# 단어를 특성벡터로 전환
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two'
])
bag = count.fit_transform(docs)

In [22]:
print(count.vocabulary_)
print(bag.toarray())

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [23]:
# tf - idf를 사용한 단어적합성
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf = True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [31]:
df.loc[0, 'review'][-50:]

"ppies only...or if you're stoned. I give this a 1."

In [34]:
import re

def preprocessor(text):
    # HTML 태그 제거
    text = re.sub('<[^>]*>', '', text)
    
    # 이모티콘 패턴 검색 (":", ";", "=" 기호로 시작하고, 그 뒤에 ")", "(", "D", "P"가 올 수 있음)
    emoticons = re.findall(r'[:;=](?:-)?[)(DP]', text)
    
    # 텍스트에서 모든 특수문자와 숫자를 공백으로 대체하고, 이모티콘은 다시 추가
    text = re.sub(r'[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', '')
    
    return text

# 테스트
print(preprocessor("</a>This :) is :( a test :-)!"))


this is a test  :) :( :)


In [36]:
df['review'] = df['review'].apply(preprocessor)
df.head(3)

,review,sentiment
0,at a saturday matinee in my home town i went w...,0
1,i love this movie it is the first film master ...,1
2,in the voice over which begins the film hughie...,1


In [37]:
# docs to token
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [39]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [44]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('runners likes running and runs a lot')[-10:] if w not in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/youngki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

In [45]:
X_train = df.loc[:25000, 'review'].values
X_test = df.loc[25000:,'review'].values
y_train = df.loc[:25000,'sentiment'].values
y_test = df.loc[25000:,'sentiment'].values

In [48]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range' : [(1,1)],
        'vect__stop_words': [None],
        'vect__tokenizer' : [tokenizer, tokenizer_porter],
        'clf__penalty' : ['l2'],
        'clf__C' : [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words' : [stop, None],
        'vect__tokenizer' : [tokenizer],
        'vect__use_idf': [False],
        'vect__norm' : [None],
        'clf__penalty':['l2'],
        'clf__C' : [1.0, 10.0]
    }
]
lr_tfidf = Pipeline([
    ('vect',tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring = 'accuracy', cv = 5, verbose = 2, n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/Users/youngki/.pyenv/versions/3.9.18/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x312c04550>; total time=   1.8s


/Users/youngki/.pyenv/versions/3.9.18/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x312c04550>; total time=   1.8s


...

[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x312c04550>, vect__use_idf=False; total time=   6.1s


/Users/youngki/.pyenv/versions/3.9.18/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x312c04550>, vect__use_idf=False; total time=   7.2s


/Users/youngki/.pyenv/versions/3.9.18/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x312c04550>,
                                              <function tokenizer_porter at 0x312c2ff70>]},
                         {'c...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x312c04550>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [49]:
# gridsearch결과 시각화
print(f"최적의 매개변수 조합 : {gs_lr_tfidf.best_params_}")
best = gs_lr_tfidf.best_estimator_
print(f"CV 정확도 {gs_lr_tfidf.best_score_:.3f}" )
print(f"테스트 정확도 : {best.score(X_test, y_test):.3f}")

최적의 매개변수 조합 : {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x312c04550>}
CV 정확도 0.895
테스트 정확도 : 0.899
